   We are using a dataset of Amazon reviews. It is compiled of three topics and contains ~200k reviews.
You will read those, convert into TF-IDF form, and use k-means clustering to analyze the results. Finally,
check the resulting clusters and see how well-defined are the clusters. The trick is that you have to
implement TF-IDF and k-means algorithms yourself!  
When implemented well, the code runs ~30 iterations in a few minutes for 10k reviews, the memory
consumption is ~2G per 10k reviews. So you probably want to work on a subset of reviews only.
Amazon Reviews
The data is downloaded from http://jmcauley.ucsd.edu/data/amazon/ and contains 3 topics: **"baby",musical instruments, and beauty**. Musical instruments is the smallest collection you won't see many
of these when inspecting the results. The original json files are transformed into csv, and contain four
variables: **date, summary, review, and rating**. In the current context you only need review as we work with
an unsupervised method.
# Explore and clean the data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import statsmodels.formula.api as smf

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
amazon=pd.read_csv("data/amazon-reviews.csv.bz2",sep="\t")

In [5]:
# save a copy of the original dataset
Amazon=amazon.copy()

In [6]:
amazon=Amazon.copy()

In [7]:
amazon.head()

,date,summary,review,rating
0,2013-07-16,Awesine,Perfect for new parents. We were able to keep ...,5
1,2013-06-29,Should be required for all new parents!,This book is such a life saver. It has been s...,5
2,2014-03-19,Grandmother watching baby,Helps me know exactly how my babies day has go...,5
3,2013-08-17,repeat buyer,I bought this a few times for my older son and...,5
4,2014-04-01,Great,I wanted an alternative to printing out daily ...,4


In [8]:
amazon.columns

Index(['date', 'summary', 'review', 'rating'], dtype='object')

In [9]:
#remove empty observations of review
amazon.drop(amazon[amazon['review']==""].index,inplace=True)
#remove all the missings
amazon.dropna(subset=['review'],inplace=True)

2. You almost certainly need to take a subset unless you run this on a beefy server. I'd recommend
to start with 1000 reviews and scale it up to 10 or 20k when the things seem to work well (requires
2-4G of RAM). Note: please take a random sample, not just first n lines. If you want to make your
results replicable, use np.random.seed.

In [10]:
np.random.seed(0)
ama=amazon.sample(1000)
ama.head()

,date,summary,review,rating
104492,2013-02-20,"Stacks Nicely, Hard to Open",I bought these to freeze some of the foods I h...,4
141042,2013-10-28,Good for homemade baby food,I make my own baby food and these work well fo...,4
117914,2012-02-23,Very happy.,"My daughter is 3 years old and 41"" tall. She s...",5
136378,2013-12-03,"Overall, good seat.",We have gone through several car seats because...,5
144640,2014-01-09,Ok for the price,We spent much more on the glider for our son's...,3


by topic? by rating? by text length?

# Implement TF-IDF transform
Here your task is the following: use the sklearn's CountVectorizer, but unlike in the Naive Bayes exer-cise, don't use binary=True option, i.e. get the actual counts, not just presence/absence of the words.  
Transform your (subset of) reviews into BOW. Next, apply TF-IDF transform on the BOW so that you end up with a TF-IDF matrix. This is what you will use later for clustering.
* transform a subset (1000 or so) reviews into BOW using sklearns CountVectorizer. Ensure you get the actual count of words, not just binary presence/absence.
* transform the BOW into tf-idf matrix. TF-IDF is a way to weigh the frequency and importance of the words, so common words get low weight and specific words, present in a subset of documents get a high weight. Consult lecture notes, section 6.1.2 about tf-idf.  

In [11]:
# ordinary BOW
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(ama.review).toarray()
words = vectorizer.get_feature_names()

In [18]:
# create a dict for the BOW column names
bowlist={}
for i in range(bow.shape[1]):
    bowlist[bow.columns[i]]=("BOW" + str(i+1))

In [34]:
bow = pd.DataFrame(data = X, columns = words)
bow=bow.T
bow.rename(bowlist,axis=1,inplace=True)

In [35]:
bow.head()

,BOW1,BOW2,BOW3,BOW4,BOW5,BOW6,BOW7,BOW8,BOW9,BOW10,...,BOW991,BOW992,BOW993,BOW994,BOW995,BOW996,BOW997,BOW998,BOW999,BOW1000
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#create a short sample for validation of results
s=bow[bow.values>1].sample(10)
s

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
and,3,1,28,1,5,5,1,1,2,1,...,0,4,3,1,0,0,3,3,0,6
she,0,0,4,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
holds,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
box,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
my,2,1,9,2,1,0,0,0,1,0,...,0,2,0,0,0,0,1,0,0,3
potty,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
his,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
on,4,1,2,0,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,1,4
him,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
the,16,6,43,2,4,10,2,0,2,3,...,9,6,1,0,3,1,7,1,0,16


In [20]:
bow.shape

(7057, 1000)

**TF-IDF：The combination of bag-of-words and cosine similarity allows to compare and classify texts.**  
$tf(x_{ij})=log(1+x_{ij})$  
**idf** assigns to words that are present in many documents low weight, and words that are in only a few documents high weight.  
$idf(j)=log\frac{N}{1+\sum_{i=1}^Ntrue(1)}$   
  
  
$tf-idf(x_{ij})=tf(x_{ij}) \bullet idf(j)$    
  
  
$\tilde x_i=(tf-idf(x_{ij}))_{j=1}^K$    
  
  
TF-IDF1 and TF-IDF2 are identical vectors and their cosine similarity is accordingly one.

In [12]:
# create a binary bow2 to calculate idf
vectorizer2 = CountVectorizer(binary=True)
Xb = vectorizer2.fit_transform(ama.review)
Xb = Xb.toarray()
bow2=pd.DataFrame(Xb)

In [21]:
N=1000

In [22]:
# calculate idf
idf=[]
for i in range(bow2.shape[1]):
    idf.append(np.log(N/(1+bow2.iloc[:,i].sum())))

len(idf)

7057

In [23]:
#define a function to calculate TF
def computeTF(bow):
    tf=[]
    
    for i in range(bow.shape[0]):
        if bow[i]!=0:
            tf.append(np.log(1+bow[i]))
        else:
            tf.append(0)
    return tf 

In [24]:
Tf=np.apply_along_axis(computeTF,1,bow)

In [25]:
TF=pd.DataFrame(Tf)
TF.shape

(7057, 1000)

In [26]:
# validate TF function
Sf=np.apply_along_axis(computeTF,1,s)
Sf=pd.DataFrame(Sf)
Sf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.386294,0.693147,3.367296,0.693147,1.791759,1.791759,0.693147,0.693147,1.098612,0.693147,...,0.000000,1.609438,1.386294,0.693147,0.000000,0.000000,1.386294,1.386294,0.000000,1.945910
1,0.000000,0.000000,1.609438,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.098612,0.693147,2.302585,1.098612,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,...,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,1.386294
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000
7,1.609438,0.693147,1.098612,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,...,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,1.609438
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2.833213,1.945910,3.784190,1.098612,1.609438,2.397895,1.098612,0.000000,1.098612,1.386294,...,2.302585,1.945910,0.693147,0.000000,1.386294,0.693147,2.079442,0.693147,0.000000,2.833213


In [27]:
# define a function to compute TF-IDF
def computeTF_IDF(u):
    ti=[]
    for i in range(TF.shape[1]):
        ti.append(u[i]*idf[i])
    return ti

In [28]:
# compute the TF-IDF of reviews
TI=np.apply_along_axis(computeTF_IDF,1,TF)
TI=pd.DataFrame(TI)

In [39]:
print(TI.shape)

(7057, 1000)


In [334]:
# the cheating sklearn function
vectorizerT = TfidfVectorizer()
Xtf = vectorizerT.fit_transform(ama.review)
Xtf = Xtf.toarray()
words = vectorizer.get_feature_names()

In [335]:
Xtf=pd.DataFrame(Xtf)
# they are different due to different definitions
Xtf.T.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Implement k-Means Clustering
The most complex step in this problem set is to implement the k-means clustering using cosine similarity
as the distance measure. Remember that k is picked by the user, not computed from the data.
I recommend to proceed as follows:
1. Pick k random reviews (remember: we work on TF-IDF matrix, so we talk about picking their
TF-IDF) and assign those to be the cluster centers.
2. assign all reviews into arbitrary clusters (all to the first is fine).
3. compute the norm of all review vectors
4. compute the norm of the cluster center vectors
5. for each cluster, compute the cosine similarity between the reviews and the corresponding cluster
centers.
Try to use vectors/matrices and numpy vectorized operations as much as you can and avoid loops.
You may easily achieve 100× speed-up in this way. Loops are fine in terms of grading, but much
slower.
6. now assign each review to the cluster, center of which is the closest to it in terms of cosine similarity.
7. did any of the reviews change cluster? If not, you are done
8. compute new cluster centers by taking mean (centroid) of all reviews that belong to each cluster.
9. repeat from 4. In my experience, you need ~30 iterations, more for more clusters.
You final result should be a nice design matrix X with condition number that is not outrageously large.
When the code runs, try to explain what exactly does it do:
1. What kind of properties of the reviews is the algorithm considering? What constitutes similar
reviews?

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
# pick a k by user
k=5

In [510]:
df=Xtf.copy()

In [511]:
#randomly pick k reviews
c=df.sample(k)

In [512]:
# assign them as cluster centers
centroids={
    i+1:c.iloc[i]
    for i in range(k)
}

In [89]:
# norm Caculator
def norm(a, ax=1):
    return np.linalg.norm(a, axis=ax)

In [409]:
# compute the norm of all review vectors & the cluster center vectors
norm_all=norm(df)
norm_center=norm(c)

In [513]:
def cs(a):
    aa=a.reshape(1,7057)
    bb=np.array(centroids[i])
    bb=bb.reshape(1,7057)
    return cosine_similarity(aa, bb, dense_output=True)


In [514]:
def assignment(df, centroids):  
    centroid_distance_cols = ['cosine_similarity{}'.format(i) for i in centroids.keys()]
    df['closest'] = df.loc[:, centroid_distance_cols].idxmax(axis=1)
    df['closest'] = df['closest'].map(lambda x: int(x.lstrip('cosine_similarity')))
    return df

In [515]:
j=0
while j<30:
    
    for i in centroids.keys():
        df['cosine_similarity{}'.format(i)]=(np.apply_along_axis(cs,1,Xtf).ravel())
    
    df = assignment(df, centroids)
    
    # update center
    centroids = update(centroids)
    c1=old_df.closest
    c2=df.closest
    dif = [c1[i] for i in range(len(c1)) if c1[i] != c2[i]]
    #test=c1.equals(c2)
    print ('iteration:', j , "difference: ", len(dif))
    j+=1
    if len(dif)==0: break    
        
        

iteration: 0 difference:  925
iteration: 1 difference:  930
iteration: 2 difference:  925
iteration: 3 difference:  924
iteration: 4 difference:  923
iteration: 5 difference:  924
iteration: 6 difference:  924


KeyboardInterrupt: 

In [439]:
old_df=df.copy()
for i in centroids.keys():
    df['cosine_similarity{}'.format(i)]=(np.apply_along_axis(cs,1,Xtf).ravel())

df = assignment(df, centroids)

In [446]:
# update center
import copy

old_centroids = copy.deepcopy(centroids)

def update(k):
    for i in centroids.keys():        
        centroids[i] = np.mean(df[df['closest'] == i].iloc[:,0:7057])
    return k

centroids = update(centroids)

In [447]:
old_df=df.copy()
for i in centroids.keys():
    df['cosine_similarity{}'.format(i)]=(np.apply_along_axis(cs,1,Xtf).ravel())

df = assignment(df, centroids)


In [443]:
c1=old_df.closest
c2=df.closest
dif = [c1[i] for i in range(len(c1)) if c1[i] != c2[i]]

In [455]:
df

,0,1,2,3,4,5,6,7,8,9,...,7053,7054,7055,7056,cosine_similarity1,cosine_similarity2,cosine_similarity3,cosine_similarity4,cosine_similarity5,closest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.256481,0.432642,0.345313,0.341398,0.320371,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.155169,0.286213,0.239814,0.225268,0.214491,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.374737,0.583459,0.472739,0.405922,0.450264,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.171546,0.256005,0.207276,0.138756,0.220157,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.256096,0.358621,0.303912,0.263688,0.319209,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.279388,0.216338,0.215406,0.217255,0.178825,1
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.214280,0.311845,0.276225,0.215648,0.271006,2
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.126911,0.178073,0.180261,0.212957,0.190094,4
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.081478,0.155075,0.107423,0.082486,0.205442,5


In [475]:
dif

[]

## sklearn.cluster.KMeans method

In [36]:
from sklearn.cluster import KMeans

In [42]:
m = KMeans(n_clusters=k).fit(TI) 
mcenter=m.cluster_centers_
yhat = m.predict(TI) 

In [45]:
hatY = m.fit(TI).labels_

In [43]:
mcenter

array([[1.50641459e+00, 1.26656077e+00, 3.26139255e+00, ...,
        3.37740835e-01, 4.28964472e-01, 3.91738925e+00],
       [6.16639827e-01, 3.02447428e-01, 1.43011573e+00, ...,
        2.91453950e-02, 1.56641385e-01, 1.40141042e+00],
       [2.50453629e-02, 1.08147131e-02, 8.14423986e-02, ...,
        2.34390514e-03, 3.14931867e-03, 4.58740485e-02],
       [8.11160359e+00, 2.73345118e+00, 9.75425785e+00, ...,
        8.01498364e-01, 1.43587936e+00, 1.23924573e+01],
       [4.05743482e+00, 2.34962077e+00, 6.50131292e+00, ...,
        2.91453950e-01, 7.83206924e-01, 5.51252972e+00]])

# Play and Analyze Your Clusters
Finally, it is time to enjoy your work ,
1. split the (subset of) reviews into clusters
2. print out a set of reviews from each cluster, so you can see and compare the results.
3. comment on what do you see. How good a job does the algorithm in picking up meaningful similarities
between reviews?  
Note: do not expect the clusters to be very clear. There will be ones that are rather obvious (all
taking about baby seats for instance) and ones that just look like a mess of all kind of things. There
are clusters that are clearly taken because some other common traits (say, talking about color,
including lipstick, baby toy, and a music player). You also need more than a few clusters (say, 15-20)
to see more meaningful content.
4. play with a few different k values. If your code works well, now it is time to scale up to as large
sample as you can (I can do 30k in ~5 mins on an oldish desktop).
5. Give some sort of final comments. How well did it work? How many clearly distinct clusters did you
see? Which k was the most interesting?

In [ ]:
cos_sim=[]
for i in centeroids.columns:        
    def cs(a):
        aa=a.reshape(1,len(a))
        #bb=np.array(centeroids[i])
        bb=bb.reshape(1,len(centeroids[i]))
        distance['distance_from_{}'.format(i)] = cosine_similarity(aa, bb, dense_output=True)
        #return cosine_similarity(aa, bb, dense_output=True)
    np.apply_along_axis(cs,1,df)



In [227]:
# make a copy of centeroids as clusters
clusters=centeroids.copy()
# assign all reviews into arbitrary clusters (all to the first is fine)
clusters[1]=df